<a href="https://colab.research.google.com/github/Arif-Kasim1/PIAIC-201/blob/main/LangChain_ToolCalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementing an example from the following url: https://python.langchain.com/docs/tutorials/agents/


In [6]:
%pip install -Uq langchain_google_genai langchain-anthropic langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

In [24]:
import getpass
import os
from google.colab import userdata
from langchain_community.tools.tavily_search import TavilySearchResults
# from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
# os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b."""
    print(f"Someone Called Me! Multiplying {a} and {b}")
    return a * b
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search, multiply]


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1735316849, 'localtime': '2024-12-27 08:27'}, 'current': {'last_updated_epoch': 1735316100, 'last_updated': '2024-12-27 08:15', 'temp_c': 15.3, 'temp_f': 59.5, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 17.4, 'wind_kph': 28.1, 'wind_degree': 172, 'wind_dir': 'S', 'pressure_mb': 1016.0, 'pressure_in': 30.0, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 75, 'feelslike_c': 15.3, 'feelslike_f': 59.5, 'windchill_c': 9.9, 'windchill_f': 49.9, 'heatindex_c': 12.1, 'heatindex_f': 53.8, 'dewpoint_c': 11.6, 'dewpoint_f': 52.9, 'vis_km': 11.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 27.2, 'gust_kph': 43.8}}"}, {'url': 'https://world-weather.info/forecas

In [13]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash-exp",
    api_key = userdata.get('GOOGLE_API_KEY')
)
response = llm.invoke([HumanMessage(content="hi!")])
response.content

'Hi there! How can I help you today?\n'

In [14]:
llm_with_tools = llm.bind_tools(tools)

response = llm_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I help you today?

ToolCalls: []


In [16]:
response = llm_with_tools.invoke([HumanMessage(content="What's the weather in SF? and 2 multiply by 8 is what?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'ecbee9e5-e925-41c3-9d4f-e4b90a49e3c2', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 2.0, 'b': 8.0}, 'id': 'da638a1b-9ce4-4f62-ac54-ff02b56b4dae', 'type': 'tool_call'}]


In [18]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='98cee715-6e0a-457d-8315-aad792b73aa2'),
 AIMessage(content='Hello! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-8cb26630-8d62-4870-ad3d-29636d6a9d4c-0', usage_metadata={'input_tokens': 122, 'output_tokens': 10, 'total_tokens': 132, 'input_token_details': {'cache_read': 0}})]

In [25]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

response = agent_executor.invoke(
    {"messages": [HumanMessage(content="what is 2 multiply by 8?")]}
)
# textwrap.fill(response['result'], width=80)
print(response["messages"])

[HumanMessage(content='what is 2 multiply by 8?', additional_kwargs={}, response_metadata={}, id='78b76cf8-0e53-4962-b890-2c5d7577d227'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 8.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-c6088c61-4716-457c-a694-82aa2b3fe930-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 8.0}, 'id': '7f811684-c1c8-4370-8d49-e6241fe65bbb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 129, 'output_tokens': 3, 'total_t

In [26]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in san francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-dff7edb9-aaa8-4db6-b407-0977c093b266-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in san francisco'}, 'id': '46a8954b-752c-4222-ab20-419895a092c4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 126, 'output_tokens': 13, 'total_tokens': 139, 'input_token_details': {'cache

{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1735315202, \'localtime\': \'2024-12-27 08:00\'}, \'current\': {\'last_updated_epoch\': 1735315200, \'last_updated\': \'2024-12-27 08:00\', \'temp_c\': 15.2, \'temp_f\': 59.4, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 11.0, \'wind_kph\': 17.6, \'wind_degree\': 251, \'wind_dir\': \'WSW\', \'pressure_mb\': 1016.0, \'pressure_in\': 30.0, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 87, \'cloud\': 75, \'feelslike_c\': 15.2, \'feelslike_f\': 59.4, \'windchill_c\': 11.4, \'windchill_f\': 52.4, \'heatindex_c\': 12.8, \'heatindex_f\': 55.0, \'dewpoint_c\': 12.7, \'dewpoint_f

In [27]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="what 2 * 5 is?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 5.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-688c3259-c5d4-40ce-beb8-a9dbc655bc54-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 5.0}, 'id': 'f577ea54-b868-4cf4-8924-e2337f110644', 'type': 'tool_call'}], usage_metadata={'input_tokens': 128, 'output_tokens': 3, 'total_tokens': 131, 'input_token_details': {'cache_read': 0}})]}}
----
{'tools': {'messages': [ToolMessage(content='10', 

{'agent': {'messages': [AIMessage(content='2 * 5 is 10.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-be40d737-2081-4efb-a99c-c18d262ebf83-0', usage_metadata={'input_tokens': 162, 'output_tokens': 10, 'total_tokens': 172, 'input_token_details': {'cache_read': 0}})]}}
----
